# Exploratory Data Analysis
Analysis of UK National Grid demand data from NESO (National Energy System Operator).

In [68]:
# Imports
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import hashlib
from datetime import datetime, timedelta
from supabase import create_client, Client
import os
from dotenv import load_dotenv

load_dotenv()

True

## Demand Data

| Column name                 | Description                                                                                              |
| --------------------------- | -------------------------------------------------------------------------------------------------------- |
| `SETTLEMENT_DATE`           | Calendar date of the electricity settlement period (UTC).                                                |
| `SETTLEMENT_PERIOD`         | Half-hour settlement period within the day (1–48).                                                       |
| `FORECAST_ACTUAL_INDICATOR` | Indicates whether the data point is **Actual (A)** or **Forecast (F)**.                                  |
| `ND`                        | **National Demand** – estimated total GB electricity demand including embedded generation.               |
| `TSD`                       | **Transmission System Demand** – demand seen by the transmission network (excludes embedded generation). |
| `ENGLAND_WALES_DEMAND`      | Electricity demand specific to England and Wales.                                                        |
| `EMBEDDED_WIND_GENERATION`  | Electricity generated by wind connected to distribution networks (not transmission).                     |
| `EMBEDDED_WIND_CAPACITY`    | Installed capacity of embedded wind generation.                                                          |
| `EMBEDDED_SOLAR_GENERATION` | Electricity generated by embedded solar PV.                                                              |
| `EMBEDDED_SOLAR_CAPACITY`   | Installed capacity of embedded solar PV.                                                                 |
| `NON_BM_STOR`               | Non-Balancing Mechanism storage output (e.g. small-scale batteries).                                     |
| `PUMP_STORAGE_PUMPING`      | Electricity demand used to pump water into pumped-storage hydro (negative net generation).               |
| `SCOTTISH_TRANSFER`         | Net electricity transfer between Scotland and England/Wales.                                             |
| `IFA_FLOW`                  | Power flow on the **IFA** interconnector (GB–France).                                                    |
| `IFA2_FLOW`                 | Power flow on the **IFA2** interconnector (GB–France).                                                   |
| `BRITNED_FLOW`              | Power flow on the **BritNed** interconnector (GB–Netherlands).                                           |
| `MOYLE_FLOW`                | Power flow on the **Moyle** interconnector (GB–Northern Ireland).                                        |
| `EAST_WEST_FLOW`            | Power flow on the **East-West** interconnector (GB–Ireland).                                             |
| `NEMO_FLOW`                 | Power flow on the **NEMO** interconnector (GB–Belgium).                                                  |
| `NSL_FLOW`                  | Power flow on the **North Sea Link** interconnector (GB–Norway).                                         |
| `ELECLINK_FLOW`             | Power flow on the **ElecLink** interconnector (GB–France).                                               |
| `VIKING_FLOW`               | Power flow on the **Viking Link** interconnector (GB–Denmark).                                           |
| `GREENLINK_FLOW`            | Power flow on the **Greenlink** interconnector (GB–Ireland).                                             |       


### Download Historic Data
Download CSV files for each year (2020-2025) from NESO data portal.

In [ ]:
# Supabase connection
supabase_url: str = os.getenv("SUPABASE_URL")
supabase_key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)
print("Connected to Supabase.")

In [53]:
# Configuration
BASE_PAGE = "https://www.neso.energy/data-portal/historic-demand-data/historic_demand_data_{}"
YEARS = range(2020, 2026)
OUT_DIR = Path("neso_historic_demand")
OUT_DIR.mkdir(exist_ok=True)

In [54]:
# HTTP session with custom User-Agent
session = requests.Session()
session.headers.update({"User-Agent": "academic-dashboard-project"})

In [55]:
# Helper function
def sha256(path: Path) -> str:
    """Calculate SHA256 hash of a file for deduplication."""
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

In [57]:
# Define possible date formats globally
date_formats = [
    '%Y-%m-%d',        # 2025-06-27
    '%d-%b-%y',        # 26-Oct-23
    '%d-%b-%Y',        # 26-Oct-2023
    '%d-%B-%y',        # 26-October-23
    '%d-%B-%Y',        # 26-October-2023
    '%Y-%m-%dT%H:%M:%S.%fZ',
    '%Y-%m-%dT%H:%M:%S',
]

In [58]:
def standardize_date(date_val):
    """Convert various date formats to a standard datetime object."""
    if pd.isna(date_val):
        return pd.NaT
    
    date_str = str(date_val).strip().upper()
    
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt.upper() if '%b' in fmt or '%B' in fmt else fmt)
        except (ValueError, TypeError):
            continue
    
    try:
        return pd.to_datetime(date_str, dayfirst=True)
    except:
        return pd.NaT

In [59]:
# Download and process data for each year
all_dfs = []
seen_hashes = set()

for year in YEARS:
    print(f"Processing {year}...")
    page_url = BASE_PAGE.format(year)

    # Fetch dataset page
    r = session.get(page_url, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Find CSV download link
    csv_url = None
    for a in soup.find_all("a", href=True):
        href = a["href"].lower()
        if "download" in href and href.endswith(".csv"):
            csv_url = a["href"]
            break

    if not csv_url:
        print(f"  No CSV found for {year}, skipping...")
        continue

    if csv_url.startswith("/"):
        csv_url = "https://www.neso.energy" + csv_url

    csv_path = OUT_DIR / f"historic_demand_{year}.csv"

    # Download CSV
    with session.get(csv_url, stream=True, timeout=60) as resp:
        resp.raise_for_status()
        with open(csv_path, "wb") as f:
            for chunk in resp.iter_content(8192):
                f.write(chunk)

    # Verify uniqueness
    file_hash = sha256(csv_path)
    if file_hash in seen_hashes:
        print(f"  Duplicate detected for {year}, skipping...")
        continue
    seen_hashes.add(file_hash)

    # Load and standardize date format
    df_year = pd.read_csv(csv_path)
    df_year['SETTLEMENT_DATE'] = df_year['SETTLEMENT_DATE'].apply(standardize_date)
    df_year["SOURCE_YEAR"] = year

    all_dfs.append(df_year)
    print(f"  Loaded {len(df_year)} records ({df_year['SETTLEMENT_DATE'].min().date()} to {df_year['SETTLEMENT_DATE'].max().date()})")

# Combine all years
df = pd.concat(all_dfs, ignore_index=True)
print(f"\nTotal records: {len(df)}")

Processing 2020...
  Loaded 17568 records (2020-01-01 to 2020-12-31)
Processing 2021...
  Loaded 17520 records (2021-01-01 to 2021-12-31)
Processing 2022...
  Loaded 17520 records (2022-01-01 to 2022-12-31)
Processing 2023...
  Loaded 17520 records (2023-01-01 to 2023-12-31)
Processing 2024...
  Loaded 17568 records (2024-01-01 to 2024-12-31)
Processing 2025...
  Loaded 16800 records (2025-01-01 to 2025-12-16)

Total records: 104496


In [60]:
# Preview the data
print(f"Columns: {df.columns.tolist()}")
print(f"Date range: {df['SETTLEMENT_DATE'].min()} to {df['SETTLEMENT_DATE'].max()}")
df.sample(5)

Columns: ['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION', 'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING', 'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW', 'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'VIKING_FLOW', 'GREENLINK_FLOW', 'SOURCE_YEAR', 'SCOTTISH_TRANSFER']
Date range: 2020-01-01 00:00:00 to 2025-12-16 00:00:00


,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SOURCE_YEAR,SCOTTISH_TRANSFER
34286,2021-12-15,15,32405,35654,30075,2568,6527,0,14001,0,...,108,375,369,-587,693,0,0,0,2021,NaN
80737,2024-08-09,4,17125,21825,16386,3448,6563,0,18053,0,...,0,-301,-526,-958,-429,159,0,0,2024,3247.0
90348,2025-02-25,13,26637,30627,25087,2087,6606,0,19726,0,...,-1071,-131,0,-632,1397,850,-1094,-439,2025,4377.0
32542,2021-11-08,47,24344,25771,23057,2649,6527,0,13915,0,...,924,122,-65,584,693,0,0,0,2021,NaN
22657,2021-04-17,4,24955,26196,22744,382,6527,0,13653,0,...,0,-19,0,975,0,0,0,0,2021,NaN


### Data Cleaning
Remove unnecessary columns and create a unified datetime column.

In [64]:
# Remove unnecessary columns
columns_to_remove = ['_id', '_full_text', '_count', 'rank', 'FORECAST_ACTUAL_INDICATOR', 'SOURCE_YEAR']
df_cleaned = df.drop(columns=[col for col in columns_to_remove if col in df.columns], errors='ignore')

print(f"Columns after cleaning: {df_cleaned.columns.tolist()}")

Columns after cleaning: ['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION', 'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING', 'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW', 'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'VIKING_FLOW', 'GREENLINK_FLOW', 'SCOTTISH_TRANSFER']


In [69]:
# Create unified DATETIME column from SETTLEMENT_DATE and SETTLEMENT_PERIOD
df_cleaned['DATETIME'] = df_cleaned.apply(
    lambda row: row['SETTLEMENT_DATE'] + timedelta(minutes=(int(row['SETTLEMENT_PERIOD']) - 1) * 30),
    axis=1
)

In [70]:
# Reorder columns and remove original date columns
cols = ['DATETIME'] + [col for col in df_cleaned.columns if col not in ['DATETIME', 'SETTLEMENT_DATE', 'SETTLEMENT_PERIOD']]
df_cleaned = df_cleaned[cols]

In [71]:

print(f"Final shape: {df_cleaned.shape}")
print(f"Date range: {df_cleaned['DATETIME'].min()} to {df_cleaned['DATETIME'].max()}")
df_cleaned.head()

Final shape: (104496, 21)
Date range: 2020-01-01 00:00:00 to 2025-12-16 23:30:00


,DATETIME,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,...,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SCOTTISH_TRANSFER
0,2020-01-01 00:00:00,26340,27153,23821,1073,6465,0,13040,0,15,...,0,852,-151,-47,854,0,0,0,0,NaN
1,2020-01-01 00:30:00,26921,27684,24393,1020,6465,0,13040,0,17,...,0,853,-146,0,854,0,0,0,0,NaN
2,2020-01-01 01:00:00,26569,27240,24085,1010,6465,0,13040,0,18,...,0,852,-53,0,854,0,0,0,0,NaN
3,2020-01-01 01:30:00,25754,26435,23350,1043,6465,0,13040,0,15,...,0,852,-66,0,854,0,0,0,0,NaN
4,2020-01-01 02:00:00,25075,25824,22788,1001,6465,0,13040,0,15,...,0,853,-74,-60,854,0,0,0,0,NaN


## Data Quality

Analyse the data quality

In [80]:
# Quick overview
print(f"{len(df_cleaned):,} records")
print(f"From {df_cleaned['DATETIME'].min()} to {df_cleaned['DATETIME'].max()}")
print(f"\n{len(df_cleaned.columns)} columns: {df_cleaned.columns.tolist()}")

104,496 records
From 2020-01-01 00:00:00 to 2025-12-16 23:30:00

21 columns: ['DATETIME', 'ND', 'TSD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION', 'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING', 'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW', 'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'VIKING_FLOW', 'GREENLINK_FLOW', 'SCOTTISH_TRANSFER']


In [81]:
# Any nulls?
missing = df_cleaned.isnull().sum()
missing = missing[missing > 0]

if len(missing) > 0:
    print("Missing values:")
    print(missing)
else:
    print("No nulls - nice!")

Missing values:
SCOTTISH_TRANSFER    52608
dtype: int64


In [ ]:
# Duplicates?
dupes = df_cleaned[df_cleaned.duplicated(subset=['DATETIME'], keep=False)]
print(f"{len(dupes)} duplicate timestamps")

if len(dupes) > 0:
    dupes.sort_values('DATETIME').head(10)

=== Duplicate Analysis ===
Duplicate timestamps: 24 (0.02%)

Sample duplicates:


,DATETIME,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,...,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SCOTTISH_TRANSFER
14350,2020-10-26 00:00:00,22276,23623,21062,2822,6527,0,13307,0,423,...,0,-171,104,504,-153,0,0,0,0,NaN
14352,2020-10-26 00:00:00,21001,22503,19849,2758,6527,0,13307,0,521,...,0,-381,88,504,257,0,0,0,0,NaN
14351,2020-10-26 00:30:00,21365,22928,20221,2829,6527,0,13307,0,586,...,0,-198,142,504,-179,0,0,0,0,NaN
14353,2020-10-26 00:30:00,21257,22840,20173,2703,6527,0,13307,0,587,...,0,-396,122,504,289,0,0,0,0,NaN
32158,2021-11-01 00:00:00,21830,23518,19727,3416,6527,0,13908,0,905,...,-155,199,101,-28,670,693,0,0,0,NaN
32160,2021-11-01 00:00:00,20538,22641,18688,3364,6527,0,13910,0,1500,...,-4,354,82,0,576,693,0,0,0,NaN
32159,2021-11-01 00:30:00,20967,23248,19018,3408,6527,0,13910,0,1498,...,-183,165,53,0,656,693,0,0,0,NaN
32161,2021-11-01 00:30:00,20724,22799,18919,3314,6527,0,13910,0,1476,...,11,360,71,0,573,673,0,0,0,NaN
49630,2022-10-31 00:00:00,20300,23810,18856,2358,6545,0,14899,0,58,...,-657,-410,265,343,-204,54,-986,0,0,NaN
49632,2022-10-31 00:00:00,18985,22446,17667,2336,6545,0,14902,0,637,...,-628,-326,399,504,47,19,-985,0,0,NaN


In [ ]:
# Check for gaps (should be 30 min intervals)
df_sorted = df_cleaned.sort_values('DATETIME')
time_diffs = df_sorted['DATETIME'].diff()
gaps = time_diffs[time_diffs > timedelta(minutes=30)]

print(f"Found {len(gaps)} gaps in the time series")

if len(gaps) > 0:
    # show the biggest ones
    gap_df = pd.DataFrame({
        'starts_after': df_sorted.loc[gaps.index - 1, 'DATETIME'].values,
        'duration': gaps.values
    }).sort_values('duration', ascending=False)
    gap_df.head(10)

=== Time Series Continuity ===
Expected interval: 30 minutes
Total gaps found: 6

Largest gaps:


,Gap Start,Gap Duration
0,2020-03-29 22:30:00,0 days 01:30:00
1,2021-03-28 22:30:00,0 days 01:30:00
2,2022-03-27 22:30:00,0 days 01:30:00
3,2023-03-26 22:30:00,0 days 01:30:00
4,2024-03-31 22:30:00,0 days 01:30:00
5,2025-03-30 22:30:00,0 days 01:30:00


In [ ]:
# Stats
df_cleaned.describe()

=== Statistical Summary ===


,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SCOTTISH_TRANSFER
count,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,104496.00,51888.00
mean,26751.33,28716.80,24514.68,1884.15,6549.37,1583.47,15890.84,0.42,192.52,783.90,251.87,276.42,-144.39,-94.51,409.27,590.20,220.77,115.49,-51.18,1810.81
std,6219.76,5931.97,5693.93,1213.40,38.49,2484.11,2550.44,10.00,362.10,1014.45,624.81,656.90,281.49,302.53,625.81,702.97,553.84,493.30,162.81,1854.79
min,12803.00,15297.00,12040.00,125.00,6465.00,0.00,13040.00,0.00,0.00,-2071.00,-1030.00,-1093.00,-505.00,-585.00,-1024.00,-1455.00,-1028.00,-1465.00,-539.00,-2851.00
25%,21876.00,24205.00,20047.00,910.00,6527.00,0.00,13721.00,0.00,7.00,0.00,-2.00,-104.00,-444.00,-374.00,0.00,0.00,0.00,0.00,0.00,282.00
50%,25865.00,27787.00,23664.00,1552.00,6545.00,6.00,15029.00,0.00,11.00,1001.00,0.00,346.00,-207.00,0.00,667.00,693.00,0.00,0.00,0.00,1664.00
75%,30672.25,32348.00,28132.00,2622.00,6562.00,2540.00,17714.00,0.00,157.00,1608.00,941.00,917.00,81.00,0.00,963.00,1395.00,871.00,0.00,0.00,3223.25
max,46433.00,47760.00,42458.00,5962.00,6622.00,14035.00,20993.00,481.00,1869.00,2041.00,1016.00,1080.00,499.00,504.00,1020.00,1419.00,1002.00,1436.00,506.00,6704.00


In [ ]:
# Outliers - using IQR on the main demand columns
for col in ['ND', 'TSD', 'ENGLAND_WALES_DEMAND']:
    if col not in df_cleaned.columns:
        continue
    q1, q3 = df_cleaned[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    outliers = df_cleaned[(df_cleaned[col] < q1 - 1.5*iqr) | (df_cleaned[col] > q3 + 1.5*iqr)]
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df_cleaned)*100:.1f}%)")

=== Outlier Detection (IQR Method) ===
ND: 236 outliers (0.23%)
  Range: [8682, 43867]
TSD: 509 outliers (0.49%)
  Range: [11990, 44562]
ENGLAND_WALES_DEMAND: 207 outliers (0.20%)
  Range: [7920, 40260]


In [ ]:
# How complete is each year? (expecting 48 periods/day × 365 days)
expected = 48 * 365

by_year = df_cleaned.groupby(df_cleaned['DATETIME'].dt.year).size()
completeness = pd.DataFrame({
    'records': by_year,
    'expected': expected,
    'pct': (by_year / expected * 100).round(1)
})
completeness

=== Data Completeness by Year ===


C:\Users\abudi\AppData\Local\Temp\ipykernel_17096\3731872549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Year'] = df_cleaned['DATETIME'].dt.year


,Records,Expected,Completeness %
Year,,,
2020,17568,17520,100.3
2021,17520,17520,100.0
2022,17520,17520,100.0
2023,17520,17520,100.0
2024,17568,17520,100.3
2025,16800,17520,95.9


In [ ]:
# Summary
nulls = df_cleaned.isnull().sum().sum()
dupes = df_cleaned.duplicated(subset=['DATETIME']).sum()

print(f"Records: {len(df_cleaned):,}")
print(f"Nulls: {nulls}")
print(f"Dupes: {dupes}")
print(f"Range: {df_cleaned['DATETIME'].min().date()} → {df_cleaned['DATETIME'].max().date()}")

if nulls == 0 and dupes == 0:
    print("\nLooks good to go")

=== Data Quality Summary ===
✓ Total records: 104,496
✓ Missing values: 52,608 (2.40%)
✓ Duplicate timestamps: 12
✓ Date range coverage: 2020-01-01 to 2025-12-16

Data quality score: Needs attention
